In [4]:
import numpy as np
import tensorflow
import random
import pandas as pd
import numpy as np
import tensorflow as tf
from tensorflow import keras
import math

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [6]:
%cd /content/drive/MyDrive/

/content/drive/MyDrive


In [7]:
MODEL_PATH = '/saved_models/nlp_models'

In [8]:
!pip install hazm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 316 kB 26.9 MB/s 
     |████████████████████████████████| 1.4 MB 57.0 MB/s 
     |████████████████████████████████| 233 kB 67.1 MB/s 
  Created wheel for nltk: filename=nltk-3.3-py3-none-any.whl size=1394485 sha256=d08d9a80a0a8baf35ed8acc26cb85ec22a98257058a9ff69d6948f7a4fa16765
  Stored in directory: /root/.cache/pip/wheels/9b/fd/0c/d92302c876e5de87ebd7fc0979d82edb93e2d8d768bf71fac4
  Created wheel for libwapiti: filename=libwapiti-0.2.1-cp37-cp37m-linux_x86_64.whl size=153884 sha256=12f73ce22a9ecb65d8e1cfbc100d9a1dea91c0c22cd49a660204becda71de928
  Stored in directory: /root/.cache/pip/wheels/ab/b2/5b/0fe4b8f5c0e65341e8ea7bb3f4a6ebabfe8b1ac31322392dbf
Successfully built nltk libwapiti
  Attempting uninstall: nltk
    Found existing installation: nltk 3.7
    Uninstalling nltk-3.7:
      Successfully uninstalled nltk-3.7


###**Text preprocessing functions**

In [9]:
from __future__ import unicode_literals
from hazm import *
import re
import random
from string import punctuation

def text_preprocess(text):
  normalizer = Normalizer()
  text = normalizer.normalize(text)
  text = re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)
  return text

def text_scramble(text):
  words = text.split()
  random.shuffle(words)
  return ' '.join(words)


###**Loading and processing Corpus 1**
(VOA FARSI 2003-08)

In [10]:
path = 'data/voa_fa_2003-2008_orig.txt'
with open(path, 'r', encoding='utf-8') as f:
  text = ' '.join([line.strip() for line in f.readlines() if not line.startswith('#')])
  text = text.split('.')
  sents = random.sample(text, 25000)

In [11]:
MAX_LEN = 170
sents_cleaned = [text_preprocess(s) for s in sents]
sents_ready = filter(lambda s: len(s) < MAX_LEN, sents_cleaned)
sents_data = list(sents_ready)
random.shuffle(sents_data)
sents_data = sents_data[:20000]  # make dataset size 20000 for ease of computation

In [12]:
tokenizer = keras.preprocessing.text.Tokenizer(oov_token='<UNK>')
tokenizer.fit_on_texts(sents_data)
tokenizer.word_index['<PAD>'] = 0
last_idx = len(tokenizer.word_index) + 1
tokenizer.word_index['<SOS>'] = last_idx

In [13]:
temp = [len(each.split()) for each in sents_data]
print('maximum sentence length:', sorted(temp)[-1])

maximum sentence length: 39


### **building the dataset**

In [14]:
def create_shuffles(sent_list, m):
  """
  this function creates m random shuffles of the sentence
  """
  all_combs = []
  for sent in sent_list:
    comb_set = set([])
    for i in range(0, m + 1):
      comb_set.add(text_scramble(sent))
    all_combs.append((sent, comb_set))
  return all_combs

In [15]:
shuffle_pairs = create_shuffles(sents_data, 8)

In [16]:
shuffle_pairs[1][0], shuffle_pairs[1][1]

(' شاهدان گفتند گلوله خانه را به آتش کشید',
 {'آتش خانه را به گفتند گلوله شاهدان کشید',
  'به آتش شاهدان گفتند کشید خانه را گلوله',
  'به را گلوله آتش خانه کشید گفتند شاهدان',
  'به گلوله آتش گفتند خانه را کشید شاهدان',
  'را شاهدان گفتند کشید گلوله آتش به خانه',
  'را گفتند خانه شاهدان آتش کشید گلوله به',
  'را گلوله به شاهدان آتش کشید خانه گفتند',
  'گلوله به خانه را کشید شاهدان آتش گفتند',
  'گلوله کشید آتش به را گفتند شاهدان خانه'})

In [17]:
def create_dataframe(pairs):
  """
  creates a dataframe with two columns of original sentence and it's shuffles
  """
  original = []
  shuffled = []
  for pair in pairs:
    for p in pair[1]:
      original.append(pair[0])
      shuffled.append(p)

  df_dict = { 'Shuffled': shuffled, 'Original': original}
  df = pd.DataFrame(df_dict)
  return df


In [18]:
df = create_dataframe(shuffle_pairs)
df = df.sample(frac=1).reset_index(drop=True)
df

,Shuffled,Original
0,از شده شرکت هیأت تاریخ برای کره نخستین در رقاب...,یک هیأت مشت زنی از کشور کره جنوبی وارد خاک کر...
1,سازند کاهش توافق برای موشکی ویژه پیشاپیش هستند...,مقامات می‌گویند دو طرف امیدوار هستند متن تواف...
2,پیدا کرده عراق در بجای که در گذاشت امروز از مذ...,خشونت مذهبی در عراق که در روزهای اخیر بیش از ...
3,اتمی آژانس تهران شورای کرده را ایران دیپلوماته...,دیپلوماتها میگویند آژانس اتمی سازمان ملل متحد...
4,شد کیفیت سازند تلفن ما روی ممنون که را می‌کنند...,ضمنا ممنون خواهیم شد اگر دوستانی که اخبار ما ...
...,...,...
177832,ازشورشیان درآوردند امنیتی توپدار کم بلوچستان ح...,نیروهای امنیتی پاکستان با حمایت هلیکوپترهای ت...
177833,میانجیگیری این پیشنهاد نخست قطرمطرح با گردیده ...,این پیشنهاد با میانجیگیری نخست وزیر قطرمطرح گ...
177834,به عملیات کشته موسوم مجروح و مقامات آمریکا دو ...,مقامات آمریکا می‌گویند درجریان عملیات موسوم ب...
177835,تا به از ۶۱ کنون نظامی گفته در جریان کشته تن س...,به گفته مقامات نظامی در جریان این حملات تا کن...


In [19]:
def create_dataset(df, train_percent=80):
  data_size = df.shape[0]
  count = math.floor(data_size * (train_percent/100))

  dataset = df['Shuffled'].values
  labels  = df['Original'].values

  train_data   = dataset[:count]
  train_labels = labels[:count]
  test_data   = dataset[count:]
  test_labels = labels[count:]

  return ((train_data, train_labels), (test_data, test_labels))




In [20]:
train_set, test_set = create_dataset(df)
X_train_text, Y_train_text = train_set
X_test_text, Y_test_text = test_set

###**making dataset ready for training**

In [21]:
sos_index = last_idx
def shift_output_sequence(seq_list):
  for seq in seq_list:
    seq.insert(0, sos_index)
  return seq_list


In [22]:
X_train_seq = tokenizer.texts_to_sequences(X_train_text)
Y_train_seq = tokenizer.texts_to_sequences(Y_train_text)
X_test_seq  = tokenizer.texts_to_sequences(X_test_text)
Y_test_seq  = tokenizer.texts_to_sequences(Y_test_text)

In [23]:
MAX_SEQ_LEN = 45  
X_train = keras.preprocessing.sequence.pad_sequences(X_train_seq, maxlen=MAX_SEQ_LEN, padding='post')
Y_train = keras.preprocessing.sequence.pad_sequences(Y_train_seq, maxlen=MAX_SEQ_LEN, padding='post')

X_test  = keras.preprocessing.sequence.pad_sequences(X_test_seq, maxlen=MAX_SEQ_LEN, padding='post')
Y_test  = keras.preprocessing.sequence.pad_sequences(Y_test_seq, maxlen=MAX_SEQ_LEN, padding='post')

X_train_decoder = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_train_seq), maxlen=MAX_SEQ_LEN, padding='post')
X_test_decoder  = keras.preprocessing.sequence.pad_sequences(shift_output_sequence(Y_test_seq),  maxlen=MAX_SEQ_LEN, padding='post')

In [24]:
X_train_decoder[:2], Y_train[:2]

(array([[24240,    11,  3791,  7906,  2027,     5,    28,    89,   183,
           165,   779,    89,   111,    24,     8,    57,    12,   218,
           220,     2,   618,     2,    11,  1146,    53, 15572,   430,
             5,    27,    28,    99,    43,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0],
        [24240,    23,    81,    27,   545,  1014,    94,  1897,   275,
          1618,   209,  1488,   600,     5,  2949,   821,     4,    34,
           686,    11,   885,  1470,   753,    12,   313,   954,  2184,
            52,     6,  2230,  1744,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32),
 array([[   11,  3791,  7906,  2027,     5,    28,    89,   183,   165,
           779,    89,   111,    24,     8,    57,    12,   218,   220,
             2,   618,     2,    11,  1146,    53, 15572,   430,     5,
            27,    28,    99,    43,    

In [25]:
X_train[:1],Y_train[:1]

(array([[    5,    24,    99,  3791,   618,    12,    89,   218,     2,
          1146,    28,   183,    53,     2,     8,    57,    11,  2027,
           111,   430,    43,    28,   220,    27,    89,   165,  7906,
           779,     5, 15572,    11,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32),
 array([[   11,  3791,  7906,  2027,     5,    28,    89,   183,   165,
           779,    89,   111,    24,     8,    57,    12,   218,   220,
             2,   618,     2,    11,  1146,    53, 15572,   430,     5,
            27,    28,    99,    43,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32))

In [27]:
VOCAB_SIZE = len(tokenizer.word_index)
VOCAB_SIZE

24240

###**Encoder-Decoder network**

In [32]:
encoder_embedding_size = 64
decoder_embedding_size = 64
lstm_units = 256

np.random.seed(42)
tf.random.set_seed(42)

encoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

encoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 1,output_dim=encoder_embedding_size, input_length=MAX_SEQ_LEN, mask_zero=True)(encoder_input)

_, encoder_state_h, encoder_state_c = keras.layers.LSTM(lstm_units, return_state=True)(encoder_embedding)

encoder_state = [encoder_state_h, encoder_state_c]

decoder_input = keras.layers.Input(shape=[None], dtype=tf.int32)

decoder_embedding = keras.layers.Embedding(input_dim=VOCAB_SIZE + 2, output_dim=decoder_embedding_size, mask_zero=True)(decoder_input)

decoder_lstm_output = keras.layers.LSTM(lstm_units, return_sequences=True)(decoder_embedding, initial_state=encoder_state)

middle_dense_1 = keras.layers.Dense(lstm_units, activation="relu")(decoder_lstm_output)

decoder_output = keras.layers.Dense(VOCAB_SIZE + 1, activation="softmax")(middle_dense_1)

model = keras.models.Model(inputs=[encoder_input, decoder_input], outputs=[decoder_output])

optimizer = keras.optimizers.Nadam(learning_rate=0.008)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
model.summary()



Model: "model_1"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_3 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 input_4 (InputLayer)           [(None, None)]       0           []                               
                                                                                                  
 embedding_2 (Embedding)        (None, None, 64)     1551424     ['input_3[0][0]']                
                                                                                                  
 embedding_3 (Embedding)        (None, None, 64)     1551488     ['input_4[0][0]']                
                                                                                            

In [33]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=20, validation_split=0.1, batch_size=512)

Epoch 1/20
251/251 [==============================] - 160s 586ms/step - loss: 2.8982 - accuracy: 0.1475 - val_loss: 2.4079 - val_accuracy: 0.2027
Epoch 2/20
251/251 [==============================] - 146s 581ms/step - loss: 1.9488 - accuracy: 0.2869 - val_loss: 1.9004 - val_accuracy: 0.2709
Epoch 3/20
251/251 [==============================] - 145s 579ms/step - loss: 1.3970 - accuracy: 0.4225 - val_loss: 1.5309 - val_accuracy: 0.3622
Epoch 4/20
251/251 [==============================] - 144s 575ms/step - loss: 1.0736 - accuracy: 0.5303 - val_loss: 1.2364 - val_accuracy: 0.4568
Epoch 5/20
251/251 [==============================] - 145s 577ms/step - loss: 0.8564 - accuracy: 0.6113 - val_loss: 1.0505 - val_accuracy: 0.5195
Epoch 6/20
251/251 [==============================] - 145s 577ms/step - loss: 0.7002 - accuracy: 0.6701 - val_loss: 0.9580 - val_accuracy: 0.5494
Epoch 7/20
251/251 [==============================] - 144s 575ms/step - loss: 0.5814 - accuracy: 0.7149 - val_loss: 0.9087 -

In [41]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=5, validation_split=0.1, batch_size=512)

Epoch 1/5
251/251 [==============================] - 145s 578ms/step - loss: 0.1703 - accuracy: 0.8996 - val_loss: 0.2775 - val_accuracy: 0.8424
Epoch 2/5
251/251 [==============================] - 144s 573ms/step - loss: 0.1419 - accuracy: 0.9155 - val_loss: 0.2942 - val_accuracy: 0.8364
Epoch 3/5
251/251 [==============================] - 144s 573ms/step - loss: 0.1531 - accuracy: 0.9090 - val_loss: 0.2358 - val_accuracy: 0.8664
Epoch 4/5
251/251 [==============================] - 144s 573ms/step - loss: 0.1342 - accuracy: 0.9195 - val_loss: 0.2454 - val_accuracy: 0.8607
Epoch 5/5
251/251 [==============================] - 143s 572ms/step - loss: 0.1450 - accuracy: 0.9135 - val_loss: 0.2602 - val_accuracy: 0.8556


In [50]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=5, validation_split=0.1, batch_size=512)

Epoch 1/5
251/251 [==============================] - 147s 585ms/step - loss: 0.1454 - accuracy: 0.9136 - val_loss: 0.2294 - val_accuracy: 0.8698
Epoch 2/5
251/251 [==============================] - 144s 573ms/step - loss: 0.1227 - accuracy: 0.9263 - val_loss: 0.2260 - val_accuracy: 0.8730
Epoch 3/5
251/251 [==============================] - 144s 572ms/step - loss: 0.1334 - accuracy: 0.9202 - val_loss: 0.2092 - val_accuracy: 0.8802
Epoch 4/5
251/251 [==============================] - 143s 569ms/step - loss: 0.1281 - accuracy: 0.9230 - val_loss: 0.2150 - val_accuracy: 0.8782
Epoch 5/5
251/251 [==============================] - 143s 570ms/step - loss: 0.1280 - accuracy: 0.9231 - val_loss: 0.2343 - val_accuracy: 0.8692


In [58]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_split=0.1, batch_size=512)

Epoch 1/10
251/251 [==============================] - 146s 580ms/step - loss: 0.1369 - accuracy: 0.9186 - val_loss: 0.2236 - val_accuracy: 0.8756
Epoch 2/10
251/251 [==============================] - 143s 572ms/step - loss: 0.1227 - accuracy: 0.9263 - val_loss: 0.1614 - val_accuracy: 0.9068
Epoch 3/10
251/251 [==============================] - 143s 569ms/step - loss: 0.1015 - accuracy: 0.9385 - val_loss: 0.1690 - val_accuracy: 0.9029
Epoch 4/10
251/251 [==============================] - 143s 569ms/step - loss: 0.1199 - accuracy: 0.9281 - val_loss: 0.2076 - val_accuracy: 0.8838
Epoch 5/10
251/251 [==============================] - 142s 568ms/step - loss: 0.1377 - accuracy: 0.9187 - val_loss: 0.1990 - val_accuracy: 0.8879
Epoch 6/10
251/251 [==============================] - 143s 569ms/step - loss: 0.1207 - accuracy: 0.9281 - val_loss: 0.1811 - val_accuracy: 0.8971
Epoch 7/10
251/251 [==============================] - 142s 567ms/step - loss: 0.1146 - accuracy: 0.9315 - val_loss: 0.1931 -

In [64]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=10, validation_split=0.1, batch_size=512)

Epoch 1/10
251/251 [==============================] - 144s 574ms/step - loss: 0.1090 - accuracy: 0.9349 - val_loss: 0.1859 - val_accuracy: 0.8960
Epoch 2/10
251/251 [==============================] - 143s 569ms/step - loss: 0.1262 - accuracy: 0.9257 - val_loss: 0.1984 - val_accuracy: 0.8913
Epoch 3/10
251/251 [==============================] - 143s 568ms/step - loss: 0.1265 - accuracy: 0.9259 - val_loss: 0.1493 - val_accuracy: 0.9149
Epoch 4/10
251/251 [==============================] - 143s 568ms/step - loss: 0.0926 - accuracy: 0.9440 - val_loss: 0.1166 - val_accuracy: 0.9318
Epoch 5/10
251/251 [==============================] - 143s 569ms/step - loss: 0.0808 - accuracy: 0.9509 - val_loss: 0.1570 - val_accuracy: 0.9109
Epoch 6/10
251/251 [==============================] - 143s 569ms/step - loss: 0.1421 - accuracy: 0.9184 - val_loss: 0.2582 - val_accuracy: 0.8661
Epoch 7/10
251/251 [==============================] - 142s 567ms/step - loss: 0.1545 - accuracy: 0.9127 - val_loss: 0.1778 -

In [66]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=5, validation_split=0.1, batch_size=512)

Epoch 1/5
251/251 [==============================] - 144s 572ms/step - loss: 0.1856 - accuracy: 0.8992 - val_loss: 0.2774 - val_accuracy: 0.8590
Epoch 2/5
251/251 [==============================] - 143s 568ms/step - loss: 0.1512 - accuracy: 0.9149 - val_loss: 0.1407 - val_accuracy: 0.9194
Epoch 3/5
251/251 [==============================] - 142s 566ms/step - loss: 0.0739 - accuracy: 0.9554 - val_loss: 0.0726 - val_accuracy: 0.9578
Epoch 4/5
251/251 [==============================] - 142s 564ms/step - loss: 0.0423 - accuracy: 0.9745 - val_loss: 0.0598 - val_accuracy: 0.9661
Epoch 5/5
251/251 [==============================] - 142s 566ms/step - loss: 0.0632 - accuracy: 0.9617 - val_loss: 0.2486 - val_accuracy: 0.8745


In [72]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=5, validation_split=0.1, batch_size=512)

Epoch 1/5
251/251 [==============================] - 144s 573ms/step - loss: 0.2615 - accuracy: 0.8685 - val_loss: 0.2872 - val_accuracy: 0.8535
Epoch 2/5
251/251 [==============================] - 143s 571ms/step - loss: 0.1347 - accuracy: 0.9233 - val_loss: 0.1010 - val_accuracy: 0.9409
Epoch 3/5
251/251 [==============================] - 142s 566ms/step - loss: 0.0469 - accuracy: 0.9721 - val_loss: 0.0489 - val_accuracy: 0.9734
Epoch 4/5
251/251 [==============================] - 142s 564ms/step - loss: 0.0268 - accuracy: 0.9849 - val_loss: 0.0382 - val_accuracy: 0.9798
Epoch 5/5
251/251 [==============================] - 142s 566ms/step - loss: 0.0359 - accuracy: 0.9785 - val_loss: 0.1474 - val_accuracy: 0.9177


In [74]:
history = model.fit([X_train, X_train_decoder], Y_train, epochs=4, validation_split=0.1, batch_size=512)

Epoch 1/4
251/251 [==============================] - 144s 572ms/step - loss: 0.3131 - accuracy: 0.8510 - val_loss: 0.3870 - val_accuracy: 0.8170
Epoch 2/4
251/251 [==============================] - 142s 568ms/step - loss: 0.1712 - accuracy: 0.9059 - val_loss: 0.1049 - val_accuracy: 0.9379
Epoch 3/4
251/251 [==============================] - 143s 568ms/step - loss: 0.0435 - accuracy: 0.9745 - val_loss: 0.0399 - val_accuracy: 0.9793
Epoch 4/4
251/251 [==============================] - 142s 566ms/step - loss: 0.0160 - accuracy: 0.9928 - val_loss: 0.0202 - val_accuracy: 0.9927


**almost 99% accuracy**

In [82]:
model.save("sentence_model_best.h5")

###**Simple seq2seq network**

**model evaluation on test data**

In [84]:
model = keras.models.load_model("sentence_model_best.h5")

In [77]:
ids = np.argmax(model.predict([X_test[:1], X_test_decoder[:1]]), axis=-1)
ids, Y_test[:1]

(array([[20958,     7,     2,  1220,   110,     2, 10594,   548,    20,
            28,   216,    17,   113,     5,  1631,   124,    33,     6,
           199,  3183,    14,     0,   101,   101,   101,   101,   101,
           101,   101,   101,   101,   101,   101,   101,   101,   101,
           101,   101,   101,   101,   101,   101,   101,   101,   101]]),
 array([[20958,     7,     2,  1220,   110,     2, 10594,   548,    20,
            28,   216,    17,   113,     5,  1631,   124,    33,     6,
           199,  3183,    14,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]],
       dtype=int32))

In [85]:
print("Evaluate on test data")
results = model.evaluate([X_test, X_test_decoder], Y_test, batch_size=512)
print("test loss, test acc:", results)

Evaluate on test data
70/70 [==============================] - 22s 289ms/step - loss: 0.0198 - accuracy: 0.9925
test loss, test acc: [0.01978108659386635, 0.9924798011779785]


###**Rebuild sentences**

In [37]:
reverse_word_map = dict(map(reversed, tokenizer.word_index.items()))
def sequence_to_text(list_of_indices):
    # Looking up words in dictionary
    words = [reverse_word_map.get(letter) for letter in list_of_indices]
    return(words)

def remove_padding_and_join(texts):
  for i in range(len(texts)):
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts
    

def reconstruct_predicted_list(texts):
  for i in range(len(texts)):
    pad_start = texts[i].index('<PAD>')
    texts[i] = texts[i][:pad_start]
    no_padding = filter(lambda a: a != '<PAD>', texts[i])
    texts[i] = ' '.join(list(no_padding))
  return texts

def get_predictions_on_test(count):
    Y_preds = model.predict([X_test[:count], X_test_decoder[:count]])
    encoded_argmax  = np.argmax(Y_preds, axis=-1)
    return encoded_argmax, Y_test[:count]

def get_predicted_sentences(predicted):
  predicted_word_list = list(map(sequence_to_text, predicted))
  predicted_sent_list = reconstruct_predicted_list(predicted_word_list)
  return predicted_sent_list

def get_original_sentences(original):
  original_word_list  = list(map(sequence_to_text, original))
  original_sent_list = remove_padding_and_join(original_word_list)
  return original_sent_list 


def rebuild_sentence_from_sequence(count=20):
  predicted, original = get_predictions_on_test(count)
  predicted_sent_list = get_predicted_sentences(predicted)
  original_sent_list  = get_original_sentences(original)
  return predicted_sent_list, original_sent_list
  


In [79]:
preds, origs = rebuild_sentence_from_sequence(10)
preds, origs

(['سیلی که در اوائل هفته در منطقهء مرکزی آن کشور جاری شد بیش از ۱۴ هزار نفر را بی خانمان کرد',
  'آنها بزرگراه\u200cها و یک گذرگاه مرزی با فرانسه را بسته بودند',
  'مقامات ارتش فیلیپین می\u200cگویند در نبردی که بین نیروهای نظامی و شورشیان کمونیست در شمال کشور درگرفت سه سرباز جانسپردند و سه نفر مجروح شدند',
  'ارتش فیلیپین می\u200cگوید در حمله نیروهای دولتی به یک موضع شورشیان در جنوب کشور دست کم ۱۳ شورشی مسلمان از پای در آمدند و ۳۰ نفر دیگر مجروح شدند',
  'پرزیدنت بوش گفت قانونگزاران باید از اشتباهاتی که موجب شکست در اصلاحات قبلی شد درس و عبرت بگیرند',
  'مقامات صنعت نفت عراق اظهارداشتند صادرات نفت خام عراق به علت حملات تروریستی وشرایط نامساعد جوی از زمان آغازجنگ تاکنون به پائین\u200cترین سطح خود تنزل کرده است',
  'خبرنگارانی که شاهد اعلام حکم دادگاه بودند میگویند بنظر میرسید صدام پس از شنیدن حکم می\u200cلرزید',
  'شوکت عزیز نخست وزیر پاکستان میگوید کشورش با بمب گذاریهای اخیر لندن و شرم الشیخ در مصر هیچگونه ارتباطی ندارد',
  'یک آتش نشان در این قسمت کشته شد و مجموعا ۲۰ نفر طی این آتش سو

In [80]:
def log(args):
  for arg in args:
    print(arg, '\n')
    print('-----\n')

In [89]:
sos_id = last_idx
def infere_sentence(sentence):
  X_text = np.asarray([sentence], dtype=object)
  X_seq = tokenizer.texts_to_sequences(X_text)
  X = keras.preprocessing.sequence.pad_sequences(X_seq, maxlen=MAX_SEQ_LEN, padding='post')
  Y_pred = np.full((X.shape[0], 1), sos_id)
  for index in range(MAX_SEQ_LEN):
    X_decoder = keras.preprocessing.sequence.pad_sequences(Y_pred, maxlen=MAX_SEQ_LEN, padding='post')
    Y_pred_next = tf.argmax(model.predict([X, X_decoder]), axis=-1)[:, index:index+1]
    #log([X, Y_pred, X_decoder, Y_pred_next])
    Y_pred = np.concatenate([Y_pred, Y_pred_next], axis=1)
  return Y_pred

def predict_a_sentence(sentence):
  Y_pred = infere_sentence(sentence)
  output = np.delete(Y_pred, np.where(Y_pred==sos_id), axis=1)
  return get_predicted_sentences(output)


test_str = 'شوکت عزیز نخست وزیر پاکستان میگوید کشورش با بمب گذاریهای اخیر لندن و شرم الشیخ در مصر هیچگونه ارتباطی ندارد'
shuffled_str = text_scramble(test_str)
print(shuffled_str)
print(predict_a_sentence(shuffled_str))
test_str = 'ایران وطن من است'
shuffled_str = text_scramble(test_str)
print(shuffled_str)
print(predict_a_sentence(shuffled_str))

بمب کشورش شوکت میگوید اخیر و لندن وزیر مصر الشیخ گذاریهای با ندارد هیچگونه شرم نخست در عزیز پاکستان ارتباطی
['شوکت عزیز نخست وزیر پاکستان میگوید کشورش با بمب گذاریهای اخیر لندن و شرم الشیخ در مصر هیچگونه ارتباطی ندارد']
من ایران وطن است
['این چیز تازه\u200cای شد']


In [90]:
path = 'data/test_sents.txt'
with open(path, 'r', encoding='utf-8') as f:
  test_sents = [line.strip() for line in f.readlines()]

MAX_LEN = 170
test_sents_cleaned = [text_preprocess(s) for s in test_sents]
len(test_sents_cleaned)

50

In [91]:
test_df = create_dataframe(create_shuffles(test_sents_cleaned, 1))
test_df

,Shuffled,Original
0,امروزه سفر کنیم که اپلیکیشن آسوده‌تر باعث سفر ...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
1,سفر شده کنیم سفر اپلیکیشن آسوده‌تر است امروزه ...,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر س...
2,فناوری است مصادیق حال شکل‌گیری افراد حال جاه‌ط...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
3,دنیای این معنی است فناوری در به آینده‌نگر مصاد...,معنی و مصادیق این واژه در حال حاضر در دنیای فن...
4,کودکان بیشتر وقت فضای می‌گذرانند در خود امروزه...,امروزه کودکان بیشتر وقت خود را در فضای مجازی م...
...,...,...
95,قضائی را کنید رفع خبرنگاران بازداشتی و مشکل محکوم,مشکل قضائی خبرنگاران محکوم و بازداشتی را رفع کنید
96,روز ویدیویی اخبار و تازه‌ترین شماست رایگان فور...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
97,و به‌صورت و اخبار رایگان در خبرهای ویدیویی دست...,تازه‌ترین خبرهای روز و اخبار فوری به‌صورت ویدی...
98,که آنها همه کسی چه کنترل رسانه‌هایی را می‌کند ...,رسانه‌هایی که همه می‌دانید چه کسی آنها را کنتر...


In [92]:
def build_predictions_dataframe(df):
  preds = []
  original = []
  for index, row in df.iterrows():
    ordered = predict_a_sentence(row['Shuffled'])
    print(row['Shuffled'])
    preds.append(ordered[0])
    original.append(row['Original'])
  new_df_dict = { 'Predicted': preds, 'Original': original}
  #print(new_df_dict)
  new_df = pd.DataFrame(new_df_dict)
  return new_df

In [93]:
result_df = build_predictions_dataframe(test_df)

امروزه سفر کنیم که اپلیکیشن آسوده‌تر باعث سفر است شده
سفر شده کنیم سفر اپلیکیشن آسوده‌تر است امروزه که باعث
فناوری است مصادیق حال شکل‌گیری افراد حال جاه‌طلب واژه در حاضر به این دنیای و دست در و آینده‌نگر در معنی
دنیای این معنی است فناوری در به آینده‌نگر مصادیق دست و در واژه در حال شکل‌گیری جاه‌طلب حال و افراد حاضر
کودکان بیشتر وقت فضای می‌گذرانند در خود امروزه مجازی را
در مجازی را بیشتر وقت می‌گذرانند فضای امروزه کودکان خود
ساده‌تر از امضای استفاده کارها می‌کند ایمیل را
امضای را ایمیل استفاده از ساده‌تر می‌کند کارها
تکاپوی در سال است از و پایانی جدید روزهای و خیلی نوشتن سال آرزوهای اهداف ما هستیم
پایانی آرزوهای جدید سال در و هستیم سال اهداف ما نوشتن خیلی است روزهای از تکاپوی و
است که فروید چند انسان شخصیت می‌گوید دارد ماندگارترین روان‌شناس ایده بزرگ جنبه شخصیت نظریه این
روان‌شناس دارد ایده نظریه انسان که چند است شخصیت شخصیت بزرگ این ماندگارترین می‌گوید فروید جنبه
مادر رقابت با تصاحب جلب می‌کند دختر رقابت مادرش احساس می‌کند پسر پدرش توجه با و پدر به‌شوق می‌کند برای و محبت
محبت دختر می‌ک

In [ ]:
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_rows', None)
result_df

,Predicted,Original
0,شانزده روز پیش آمده است بار دیگر جزو بقتل رساند,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
1,شانزده بار از مردی که روز دوشنبه شده بود بدان,اپلیکیشن سفر باعث شده است که امروزه آسوده‌تر سفر کنیم
2,در همین حال طی آن اعلام منابع نفت برچیده شده است اما نگفته‌اند بعضی از زخمی شدن جمعیت جهان را یافته‌اند بودند,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
3,در همین حال به این ترتیب توسط یک حقوق بشر می‌گوید بیش از ۱۴۰۰ نفر زودتر سال گذشته نیز کسی است,معنی و مصادیق این واژه در حال حاضر در دنیای فناوری به دست افراد جاه‌طلب و آینده‌نگر در حال شکل‌گیری است
4,ماموران امنیتی عراق از فاجعه حکم دادگاه نیاز دارد دارند,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
5,ماموران امنیتی عراق به نیویورک معیارهای را مورد درمان قراردهند,امروزه کودکان بیشتر وقت خود را در فضای مجازی می‌گذرانند
6,این کار در پی هم شدت خوبی هستند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
7,این کار که همزمان با سقوط حضور دارند,استفاده از امضای ایمیل کارها را ساده‌تر می‌کند
8,ما و دختر مک روحیه تغییر بر این است که باید پدر این برنامه را بر عهده نمیکند,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
9,ما و دختر مک روحیه تغییر بر این است که باید پدر این برنامه را بر عهده میکند,روزهای پایانی سال است و خیلی از ما در تکاپوی نوشتن اهداف و آرزوهای سال جدید هستیم
